# Hate Speech Detection on Dynabench

[![GitHub stars](https://img.shields.io/github/stars/eugenesiow/practical-ml?style=social)](https://github.com/eugenesiow/practical-ml/stargazers)          [![Twitter](https://img.shields.io/twitter/url?url=https%3A%2F%2Fgithub.com%2Feugenesiow%2Fpractical-ml)](https://twitter.com/intent/tweet?text=Wow:&url=https%3A%2F%2Fgithub.com%2Feugenesiow%2Fpractical-ml)          [![GitHub license](https://img.shields.io/github/license/eugenesiow/practical-ml)](https://github.com/eugenesiow/practical-ml/blob/master/LICENSE)

---



Notebook to train an RoBERTa model to perform hate speech detection. The [dataset](https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset) used is the Dynabench Task - Dynamically Generated Hate Speech Dataset from the paper by [Vidgen et al. (2020)](https://arxiv.org/abs/2012.15761). 

> The dataset provides 40,623 examples with annotations for fine-grained labels, including a large number of challenging contrastive perturbation examples. Unusually for an abusive content dataset, it comprises 54% hateful and 46% not hateful entries.

There is no published state-of-the-art model on this dataset at this point though the task on Dynabench reports mean MER (Model Error Rate) scores. We are able to train the model to a F1 score of **86.6%** after only 1 epoch.

The notebook is structured as follows:
* Setting up the GPU Environment
* Getting Data
* Training and Testing the Model
* Using the Model (Running Inference)

## Task Description

> Hate Speech Detection is the automated task of detecting if a piece of text contains hate speech.

# Setting up the GPU Environment

#### Ensure we have a GPU runtime

If you're running this notebook in Google Colab, select `Runtime` > `Change Runtime Type` from the menubar. Ensure that `GPU` is selected as the `Hardware accelerator`. This will allow us to use the GPU to train the model subsequently.

#### Install Dependencies and Restart Runtime

In [1]:
!pip install -q transformers
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 124.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

You might see the error `ERROR: google-colab X.X.X has requirement ipykernel~=X.X, but you'll have ipykernel X.X.X which is incompatible` after installing the dependencies. **This is normal** and caused by the `simpletransformers` library.

The **solution** to this will be to **reset the execution environment** now. Go to the menu `Runtime` > `Restart runtime` then continue on from the next section to download and process the data.

# Getting Data

#### Pulling the data from Github

The code below uses `pandas` to pull the dataset as a CSV file from the [official Github repository](https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset). The dataset is now stored as a  dataframe, in which we can transform for use with the `simpletransformers` library to train the model. So we pull the CSV from Github and split the CSV into training and test sets by using the column `split` in the CSV file which indicates which example/sample is a training sample and which is a test sample.

In [8]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.2.csv')
train_df = df[df['split'] == 'train'] # split the dataset by the column 'split' which labels 'train' and 'test' samples
test_df = df[df['split'] == 'test'] # split the dataset by the column 'split' which labels 'train' and 'test' samples

Once done we can take a look at the `head()` of the training set to check if our data has been retrieved properly.

In [4]:
train_df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,hate,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


In [9]:
train_df["label"].unique()

array(['hate', 'nothate'], dtype=object)

We transform the dataframe column `label` so that the labels `hate` and `nothate` are now integers `1` and `0` respectively. This input format of labels is required for our training step with the `transformers` library.

In [10]:
train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
train_df.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,1,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,1,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,1,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
14,15,acl1001,15,It's such a rape culture with these immigrants...,1,notgiven,notgiven,original,train,1,CAgNlUizNm,1,NaN


We also rename the `label` column to `labels` as this also conforms to the input format required for the `simpletransformers` library.

In [11]:
train_df = train_df.rename(columns={'label': 'labels'})
test_df = test_df.rename(columns={'label': 'labels'})

We can now take a look at the train and test set sizes. We see that this dataset is quite special as the `hate` and `nothate` class sizes are actually quite close in proportion.

In [12]:
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]], 
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,15202,1853
1,17804,2271


# Training and Testing the Model

#### Set the Hyperparmeters

First we setup the hyperparamters. We train to 1 epoch only as want the training to complete fast. The important parameters are the `max_seq_length`, which we set to `64` and `sliding_window` to true. As we don't have a high RAM GPU on Colab we can't set the `max_seq_length` to too large a value and by using `sliding_window` we at least are able to handle longer sequences without truncation. See the [simpletransformers documentation](https://simpletransformers.ai/docs/classification-specifics/#training-with-sliding-window) for a more detailed explanation of a sliding window.

In [13]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

#### Train the Model

Once we have setup the hyperparemeters in the `train_args` dictionary, the next step would be to train the model. We use the RoBERTa model from the awesome [Hugging Face Transformers](https://github.com/huggingface/transformers) library and use the [Simple Transformers library](https://simpletransformers.ai/docs/classification-models/) on top of it to make it so we can train the classification model with just 2 lines of code.

RoBERTa is an optimized BERT model by Facebook Research with better performance on the masked language modeling objective that modifies key hyperparameters in BERT, including removing BERT’s next-sentence pretraining objective, and training with much larger mini-batches and learning rates. In short, its a bigger but generally better performing BERT model we can easily plug in here with the transformers library.

In [14]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args) 

# Train the model, there is no development or validation set for this dataset 
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/33006 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/299 [00:00<?, ?it/s]

  0%|          | 0/4124 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/588 [00:00<?, ?it/s]

The model has been trained and evaluating on the test set after training to only 1 epoch gives an accuracy of **85.9%**. We want to also evaluate the F1 score which is a better measure as the dataset is slightly imbalanced.

In [15]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.f1_score)

  0%|          | 0/4124 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/588 [00:00<?, ?it/s]

We see that the output F1 score from the model after training for 1 epoch is **86.6%** ('acc': 0.8661675245671503).

# Using the Model (Running Inference)

Running the model to do some predictions/inference is as simple as calling `model.predict(input_list)`.

In [17]:
samples = [test_df[test_df['labels'] == 0].sample(1).iloc[0]['text']] # get a random sample from the test set which is nothate
predictions, _ = model.predict(samples)
label_dict = {0: 'nothate', 1: 'hate'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 - nothate: It's not a craze but makes people who transition or encourage it a hero to their friends. A craze is something that makes American for-profit medical clinics a lot of money


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:2253: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_pred, counts = mode(pred_row)


In [43]:
from bs4 import BeautifulSoup
# function to remove HTML tags
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

In [40]:
text='''<turnaway-pharos-heading no-margin="" class="title-font" level="1" preset="5" data-pharos-component="PharosHeading">
            ISLAMIC TERRORISM IN INDIA AND ITS IMPACT ON MUSLIM COMMUNITY AND HUMAN RIGHTS
        </turnaway-pharos-heading>'''

In [45]:
html_removed_text=remove_html_tags(text)
html_removed_text

'\n            ISLAMIC TERRORISM IN INDIA AND ITS IMPACT ON MUSLIM COMMUNITY AND HUMAN RIGHTS\n        '

In [46]:
predictions, _ = model.predict([html_removed_text])
label_dict = {0: 'nothate', 1: 'hate'}
print(label_dict[predictions[0]])


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

nothate


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:2253: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_pred, counts = mode(pred_row)


In [32]:
import pickle

We can also generate a `results.txt` file from the test set. The file is stored in our Colab environment. Hit the `folder` icon at the side and you can download the `results.txt` file from the file browser. You can submit this `.txt` file to [Dynabench](https://dynabench.org/tasks/5#overall) for evaluation if you wish to.

In [20]:
predictions, _ = model.predict(test_df['text'].tolist())
df = pd.DataFrame(predictions)
df.to_csv('results.txt', index=False, header=False) # saves the prediction results to a file in the colab environment

  0%|          | 0/4124 [00:00<?, ?it/s]

  0%|          | 0/588 [00:00<?, ?it/s]

We can connect to Google Drive with the following code to save any files you want to persist. You can also click the `Files` icon on the left panel and click `Mount Drive` to mount your Google Drive.

The root of your Google Drive will be mounted to `/content/drive/My Drive/`. If you have problems mounting the drive, you can check out this [tutorial](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


You can move the model checkpount files which are saved in the `/content/outputs/best_model/` directory to your Google Drive.

In [31]:
import shutil
shutil.move('/outputs/', "/content/drive/My Drive/outputs/")

'/content/drive/My Drive/outputs/'

More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml) and do drop us some feedback on how to improve the notebooks on the [Github repo](https://github.com/eugenesiow/practical-ml/).